In [1]:
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

# Spark context - Spark Session
It allows Spark Driver to access the cluster through its Cluster Resource Manager and can be used to create RDDs, accumulators and broadcast variables on the cluster. 
Spark Context also tracks executors in real-time by sending regular heartbeat messages.

In [3]:
spark = (SparkSession.builder 
        .master("local") 
        .appName("workshop_spark")
        #.option("spark.driver.bindAddress","localhost:4040")
        .getOrCreate())

In [4]:
spark

In [29]:
municipios = spark.read.option("header", True).csv("data/municipios.csv")
populacao = spark.read.option("header", True).csv("data/populacao_municipios.csv")
nomes = spark.read.option("header", True).csv("data/nomes_municipios.csv")

In [17]:
municipios.show(5)

+------------+--------------+----------------+---------------+----------------+------------------+--------------+----------+---------------+--------------+------------------+---------------+-----------------------+--------------------+---------------+-------------+--------------+-----------------+---------+------------+--------+------+------------+------------+------------+
|id_municipio|id_municipio_6|id_municipio_TSE|id_municipio_RF|id_municipio_BCB|         municipio|capital_estado|id_comarca|id_regiao_saude|  regiao_saude|id_regiao_imediata|regiao_imediata|id_regiao_intermediaria|regiao_intermediaria|id_microrregiao| microrregiao|id_mesorregiao|      mesorregiao|id_estado|estado_abrev|  estado|regiao|existia_1991|existia_2000|existia_2010|
+------------+--------------+----------------+---------------+----------------+------------------+--------------+----------+---------------+--------------+------------------+---------------+-----------------------+--------------------+-----------

In [18]:
municipios.columns

['id_municipio',
 'id_municipio_6',
 'id_municipio_TSE',
 'id_municipio_RF',
 'id_municipio_BCB',
 'municipio',
 'capital_estado',
 'id_comarca',
 'id_regiao_saude',
 'regiao_saude',
 'id_regiao_imediata',
 'regiao_imediata',
 'id_regiao_intermediaria',
 'regiao_intermediaria',
 'id_microrregiao',
 'microrregiao',
 'id_mesorregiao',
 'mesorregiao',
 'id_estado',
 'estado_abrev',
 'estado',
 'regiao',
 'existia_1991',
 'existia_2000',
 'existia_2010']

-usar partition (pq fazer)
-fazer caching (reduz o custo de io)
-join
-mostrar o explain()?
-usar udf (quais os perigos) - Use the higher-level standard Column-based functions with
Dataset operators whenever possible before reverting to
using your own custom UDF functions since UDFs are a
blackbox for Spark and so it does not even try to optimize them.
-window?
-escrever parquet com particionamento

-no final rodar em nuvem

## Particionamento
https://luminousmen.com/post/spark-partitions
https://luminousmen.com/post/spark-tips-partition-tuning

- Particionamento é a principal unidade de paralelismo no Apache Spark.
- Cada partição é enviada para workers
- artições de mais ou de menos podem afetar performance:
    - Partição de menos: Não utiliza bem os recursos do cluster
    - Partições de mais: Introduz overhead no gerenciamento de muitas partições

-----

And manual partitioning is important for proper data balancing between partitions. As a rule, smaller partitions allow us to distribute data more evenly, among more executors. Therefore, smaller partitions can boost tasks with more repartitions (data reorganization during computation).


In [28]:
print(f'Particionamento padrão: {municipios.rdd.getNumPartitions()}')
repartition = municipios.repartition(10)
print(f'Particionamento "aleatório": {repartition.rdd.getNumPartitions()}')
repartition = municipios.repartition('id_municipio')
print(f'Particionamento com chave de Negócio: {repartition.rdd.getNumPartitions()}')

Particionamento padrão: 1
Particionamento "aleatório": 10
Particionamento com chave de Negócio: 200


In [ ]:
#Salvar parquet

## Cache
https://towardsdatascience.com/apache-spark-caching-603154173c48

Caching is beneficial when we use particular RDD several times (and can slow down our calculations otherwise where the whole lineage graph will be processed several times).
Como funciona? O dataset fica em memória em todos os workers

---> broadcast

In [ ]:
populacao.cache

In [ ]:
#filtrar populacao para 2010
#filtrar municipios para os que existiam em 2010

#Join
df = (nomes.join(municipios, "id_municipio")
          .join(populacao, "id_municipio"))

## UDF

In [ ]:
def squared(s):
    return s + s
squared2 = spark.udf.register("squaredWithPython", squared, LongType())
df_sp.withColumn("square", squared2(col("Confirmed").cast(IntegerType()))).show()

## Windows

-----
-Fazer janela de dado temporal para pegar o lag() como coluna de valor anterior
-Pegar valor máximo dos últimos 10 anos

## Execution plan
- Transform: operações de transformações dos dados do RDD, que resultam em um novo RDD.lazy que não são executadas naquele momento
    - narrow: não precisam de dados de outras partições (map)
    - wide: precisam de shuffle (groupby)
- Ações: operações que precisam avaliar o dataframe como um todo (show(), write()). É quando ele executa o plano

--------
chamar explain, fazer operação, chamar explain, fazer show(), chamar explain

In [ ]:
df.explain(True)

## Checkpoint
https://medium.com/@adrianchang/apache-spark-checkpointing-ebd2ec065371

## Referências

- Série inicial: https://luminousmen.com/post/hadoop-yarn-spark
- Série 2: https://luminousmen.com/post/spark-tips-dataframe-api
- https://www.youtube.com/watch?v=daXEp4HmS-E&feature=youtu.be